# Example usage of .pyjama format

In [9]:
import glob
import sys
import ipdb
import os
import numpy as np
import json

import pyjama
import importlib
importlib.reload(pyjama);


In [10]:
def F_readCsv2(annotFile):
    """  annotTime_l, annotLabel_l = F_readCsv2(annotFile) """
    with open(annotFile, 'r') as text_file:
        lines = text_file.readlines()
    annotTime_l = [ float(line.strip('\n').split('\t')[0]) for line in lines]
    annotLabel_l = [ line.strip('\n').split('\t')[1] for line in lines]
    return annotTime_l, annotLabel_l

In [11]:
def F_readCsv3(annotFile):
    """ annotTime_l, annotStop_l, annotLabel_l = F_readCsv3(annotFile) """
    with open(annotFile, 'r') as text_file:
        lines = text_file.readlines()
    annotTime_l = [ float(line.strip('\n').split('\t')[0]) for line in lines]
    annotStop_l = [ float(line.strip('\n').split('\t')[1]) for line in lines]
    annotLabel_l = [ line.strip('\n').split('\t')[2] for line in lines]
    return annotTime_l, annotStop_l, annotLabel_l

## Simple dummy example

In [12]:
def F_example_segment_pitch_and_text(pyjama_file):
    """
    Example of using pyjama to the store the structural segments of a
    collection (here we consider the SALAMI music structure collection)
    """

    # --- 1) Create the pyjama file structure
    #myPyjama = pyjama.C_pyjama(not_valid_action='reject')
    myPyjama = pyjama.C_pyjama(not_valid_action='add_to_dictionary')
    #myPyjama = pyjama.C_pyjama(not_valid_action='filter_out')

    myPyjama.M_add_definition(description_name='filepath',
                             type_constraint='free')
    myPyjama.M_add_definition(description_name='pitchandtexttype',
                             type_extent='segment',
                             type_constraint='free',
                             generator={},
                             dictionary=[])


    myPyjama.M_add_entry()
    myPyjama.M_update_entry(description_name='filepath', value_l='./myAudioFile.mp3')
    myPyjama.M_update_entry(description_name='pitchandtexttype', value_l=[440., 'wasabi'], time_l=0.0, duration_l=1.0)

    # --- 3) Save
    myPyjama.M_save(pyjama_file)
    return

pyjama_file = './_examples/example_segment_pitch_and_text.pyjama'
F_example_segment_pitch_and_text(pyjama_file)

writting pyjama file: ./_examples/example_segment_pitch_and_text.pyjama


## Storing file structure

In [13]:
importlib.reload(pyjama) 

def F_example_segment(pyjama_file):
    """
    Example of using pyjama to the store the structural segments of a
    collection (here we consider the SALAMI music structure collection)
    """

    # --- 1) Create the pyjama file structure
    #myPyjama = pyjama.C_pyjama(not_valid_action='reject')
    myPyjama = pyjama.C_pyjama(not_valid_action='add_to_dictionary')
    #myPyjama = pyjama.C_pyjama(not_valid_action='filter_out')
    myPyjama.M_add_definition(description_name='filepath',
                             type_constraint='filepath')
    myPyjama.M_add_definition(description_name='structtype',
                             type_extent='segment',
                             type_constraint='value_in_dictionary',
                             generator={'name': '', 'version': '', 'date': ''},
                             dictionary=['Silence', 'A', 'B', 'C', 'D', 'E', 'F', 'I', 'V', 'End'])

    myPyjama.M_add_definition(description_name='structtypeList',
                             type_extent='segment',
                             type_constraint='value_in_dictionary',
                             generator={'name': '', 'version': '', 'date': ''},
                             dictionary=['Silence', 'A', 'B'])

    audioFile_l = glob.glob('./_examples/*.mp3')

    for audioFile in audioFile_l:
        annotFile = audioFile.replace('.mp3', '.struct.lab')
        if os.path.isfile(annotFile):
            # --- 2) Create a new entry and update its value
            myPyjama.M_add_entry()
            myPyjama.M_update_entry(description_name='filepath', value_l=audioFile)
            annotTime_l, annotStop_l, annotLabel_l = F_readCsv3(annotFile)
            annotDuration_l = []
            for num in range(0, len(annotTime_l)):
                myPyjama.M_update_entry(description_name='structtype', value_l=annotLabel_l[num], time_l=annotTime_l[num], duration_l=annotStop_l[num]-annotTime_l[num])
                annotDuration_l.append(annotStop_l[num]-annotTime_l[num])

            myPyjama.M_update_entry(description_name='structtypeList', value_l=[annotLabel_l], time_l=annotTime_l, duration_l=annotDuration_l)

    # --- 3) Save
    myPyjama.M_save(pyjama_file)
    return

pyjama_file = './_examples/example_segment.pyjama'
F_example_segment(pyjama_file)

'value'(C) is not part of dictionary ->  updating dictionary
writting pyjama file: ./_examples/example_segment.pyjama


## Updating dictionary



In [14]:
importlib.reload(pyjama) 

def F_example_updating_dictionary(pyjama_file):
    """
    This is an example of code that store genre tags and automatically update a dictinary 
    """
    # --- 1) Create the pyjama file structure
    my = pyjama.C_pyjama()

    # --- 2) Create a first entry and update its value
    my.M_add_definition(description_name='genre', type_extent='global', type_constraint='value_in_dictionary')

    tag_l = ['blues', 'blues', 'hiphop', 'rock', ['blues', 'hiphop']]
    for tag in tag_l:
        my.M_add_entry()
        my.M_update_entry(description_name='genre', value_l=tag)

    # --- 3) Save
    my.M_save(pyjama_file)
    return

pyjama_file = './_examples/example_updating_dictonary.pyjama'
F_example_updating_dictionary(pyjama_file)

'value'(blues) is not part of dictionary ->  updating dictionary
'value'(hiphop) is not part of dictionary ->  updating dictionary
'value'(rock) is not part of dictionary ->  updating dictionary
writting pyjama file: ./_examples/example_updating_dictonary.pyjama


## Storing breakpoint values

In [15]:
importlib.reload(pyjama) 

def F_example_breakpoint(pyjama_file):
    """
    This is an example of code to store a 'breakpoint' function in pyjama
    In a breakpoint
        times must be represented as a numpy vector (not a matrix) with dimension (nbDim)
        values must be represented as a numpy matrix with dimensions (nbDim, nbTime)
    Pyjama store both as list or list of lists
    """


    # --- 1) Create the pyjama file structure
    my = pyjama.C_pyjama()

    # --- 2) Create a first entry and update its value
    my.M_add_definition(description_name='f0', type_extent='breakpoint', row_name=['f0'])
    my.M_add_definition(description_name='mfcc', type_extent='breakpoint', row_name=['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13'])
    my.M_add_definition(description_name='meanMfcc', type_extent='global', type_content='numeric', row_name=['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13'])

    my.M_add_entry()

    # --- Many Time, One dimension
    if True:
        f0_time_v = np.arange(0, 1, 0.01)
        nb_time = f0_time_v.shape[0]
        f0_value_m = np.random.randn(1, nb_time)
        my.M_update_entry(description_name='f0', time_l=f0_time_v, value_l=f0_value_m)

    # --- Many Time, Many dimension
    if True:
        mfcc_time_v = np.arange(0, 1, 0.01)
        mfcc_value_m = np.random.randn(13, mfcc_time_v.shape[0])
        my.M_update_entry(description_name='mfcc', time_l=mfcc_time_v, value_l=mfcc_value_m, )

    if True:
        mfcc_time_v = np.arange(0, 1, 0.01)
        mfcc_value_m = np.random.randn(13, mfcc_time_v.shape[0])
        my.M_update_entry(description_name='meanMfcc', value_l=np.mean(mfcc_value_m, axis=1, keepdims=True))

    # --- 3) Save
    my.M_save(pyjama_file)
    return

pyjama_file = './_examples/example_breakpoint.pyjama'
F_example_breakpoint(pyjama_file)

writting pyjama file: ./_examples/example_breakpoint.pyjama


In [16]:
# --- Re-load to pyjama file and convert back the breakpoint to numpy format
with open(pyjama_file) as f:
    data_d = json.load(f)

time_v = np.array(data_d['collection']['entry'][0]['f0'][0]['time'])
value_m = np.array(data_d['collection']['entry'][0]['f0'][0]['value'])
print(time_v.shape)
print(value_m.shape)

time_v = np.array(data_d['collection']['entry'][0]['mfcc'][0]['time'])
value_m = np.array(data_d['collection']['entry'][0]['mfcc'][0]['value'])
print(time_v.shape)
print(value_m.shape)

(100,)
(1, 100)
(100,)
(13, 100)


## Storing breakpoint time/value

In [17]:
importlib.reload(pyjama) 

def F_example_breakPoint_time_value(pyjama_file):
    """
    This an example to store several 'breakpoint' functions with a shared time
    In this case the times are stored as a 'breakpointTime'
    while the values of each breakpoint are stored as 'breakpointValue'?
    in a 'breakpointTime'
        times must be represented as a numpy vector (not a matrix) with dimension (nbDim)
    in a 'breakpointValue'
        values must be represented as a numpy matrix with dimensions (nbDim, nbTime)
    """

    # --- 1) Create the pyjama file structure
    my = pyjama.C_pyjama()
    my.M_add_definition(description_name='shared-time', type_extent='breakpoint_time')
    my.M_add_definition(description_name='alldescriptors', type_extent='breakpoint_value', row_name=['f0', 'harmonicity'])
    my.M_add_definition(description_name='loudness', type_extent='breakpoint_value', row_name=['loudness'])

    nb_time = 10
    # --- 2) Create a new entry and update its value
    my.M_add_entry()
    time_v = np.arange(0,nb_time).reshape((1,nb_time)) # --- for breakpoint_time, the time is considered as a value it should therefore be a matrix (ndim=2)
    my.M_update_entry(description_name='shared-time', value_l=time_v)
    my.M_update_entry(description_name='alldescriptors', value_l=np.zeros((2, nb_time)))
    my.M_update_entry(description_name='loudness', value_l=np.zeros((1, nb_time)))

    # --- 3) Save
    my.M_save(pyjama_file)

    return

pyjama_file = './_examples/example_breakpoint_time_value.pyjama'
F_example_breakPoint_time_value(pyjama_file)

writting pyjama file: ./_examples/example_breakpoint_time_value.pyjama


In [18]:
# --- Re-load to pyjama file and convert back the breakpoint to numpy format
with open(pyjama_file) as fid:
    data_d = json.load(fid)

time_v = np.array(data_d['collection']['entry'][0]['shared-time'][0]['value'])
alldescriptor_m = np.array(data_d['collection']['entry'][0]['alldescriptors'][0]['value'])
loudness_v = np.array(data_d['collection']['entry'][0]['loudness'][0]['value'])

print(time_v.shape)
print(alldescriptor_m.shape)
print(loudness_v.shape)

(1, 10)
(2, 10)
(1, 10)
